# Prueba Técnica

### Laura Camila Rincón Manrique

In [ ]:
#pip install requests
#!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import json
from datetime import datetime, timedelta
import os

In [2]:
def Get_data_dates(url,date):
    url = f"{url}{date}"
    api_response = requests.get(url)
    if api_response.status_code == 200:
        return api_response.json()
    else:
        print(f"Error al obtener los datos para la fecha {date}, Error API: {api_response.status_code}")
        return None

In [3]:
def Verify_file_exist(path_file):   
    if os.path.exists(path_file):
        os.remove(path_file)
        print(f"The file {path_file} was delete.")
    else:
        print(f"The file {path_file} doesn't exist and It is goint to create.")

In [4]:
def Save_json(data,path_file):
    with open(path_file, 'w') as archivo:
        json.dump(data, archivo, indent=4)
    print(f"Datos almacenados en {path_file}")

In [5]:
#Data Range
end_date = datetime(2024, 1, 31) 
initial_date = datetime(2024, 1, 1)
actual_date = initial_date
url = "http://api.tvmaze.com/schedule/web?date="
archivo = 'ejemplo.json'


list_data = []

while actual_date <= end_date:
    date_str = actual_date.strftime('%Y-%m-%d')  #YYYY-MM-DD
    path_file = f'C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_{date_str}.json'
    print(f"Getting the data to: {date_str}")

    data = Get_data_dates(url,date_str)

    if data:
        list_data.extend(data)
        Verify_file_exist(path_file)
        Save_json(data,path_file)

    actual_date += timedelta(days=1)



Getting the data to: 2024-01-01
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-01.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-01.json
Getting the data to: 2024-01-02
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-02.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-02.json
Getting the data to: 2024-01-03
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-03.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-03.json
Getting the data to: 2024-01-04
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-04.json was delete.
Datos almacenados en C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-04.json
Getting the data to: 2024-01-05
The file C:/Users/camil/Desktop/prueba_tecnica_lulo/json/data_2024-01-05.json was delete.
Datos almacenados en C:/Users/

4807